In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

df_train['Source'] = 'train'
df_test['Source'] = 'test'
df_test['Item_Outlet_Sales'] = 0

In [3]:
df = pd.concat([df_train,df_test],axis=0,ignore_index=True)
print(df_train.shape)
print(df_test.shape)
print(df.shape)
df.head(2)

(8523, 13)
(5681, 13)
(14204, 13)


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,Source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train


##  Part 1: Data Cleansing and Preparation

### 1. Handling casing errors in Fat Content feature

In [4]:
def prepare_item_fat_content(df):
    df['Item_Fat_Content'] = df['Item_Fat_Content'].astype(str)
    df['Item_Fat_Content'] = np.where(df.Item_Fat_Content == 'low fat','Low Fat',df.Item_Fat_Content)
    df['Item_Fat_Content'] = np.where(df.Item_Fat_Content == 'LF','Low Fat',df.Item_Fat_Content)
    df['Item_Fat_Content'] = np.where(df.Item_Fat_Content == 'reg','Regular',df.Item_Fat_Content)
    return df

### 2. Handling missing values in Item Weight feature

In [5]:
def fill_item_weights(df):
    item_avg_weight = df.pivot_table(values='Item_Weight', index='Item_Identifier')
    miss_bool = df['Item_Weight'].isnull()
    df.loc[miss_bool,'Item_Weight'] = df.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight.loc[x])
    return df

### 3. Finding Outlet Age

In [6]:
def find_outlet_age(df):
    df['Outlet_Age'] = 2013 - df['Outlet_Establishment_Year']
    df.drop('Outlet_Establishment_Year',axis=1,inplace=True)
    return df

### 4. Handling Missing Values in Outlet Size

In [7]:
def prepare_outlet_size(df):
    df['Outlet_Size'] = np.where(df.Outlet_Type == 'Grocery Store','Small',df.Outlet_Size)
    df['Outlet_Size'] = np.where((df.Outlet_Type == 'Supermarket Type1') & (df.Outlet_Location_Type == 'Tier 2'),'Small',df.Outlet_Size)
    return df

### 5. Adding Item type feature

In [8]:
def add_item_type(df):
    df['Item_Type_Derived'] = df['Item_Identifier'].apply(lambda x: x[0:2])
    df['Item_Type_Derived'] = df['Item_Type_Derived'].map(
    {
        'FD':'Food',
        'NC':'Non-Consumables',
        'DR':'Drinks'
    })
    return df

### 6. Converting Dummy Variables

In [9]:
def encode_variables(df,columns):
    df_dummies = pd.get_dummies(df[columns])
    df = df.drop(columns,axis=1)
    return pd.concat([df,df_dummies],axis=1)

### 7. Convert Ordinal Variables

In [10]:
def convert_ordinal_variables(df, columns):
    le = LabelEncoder()
    df_cols = df[columns]
    cols = df_cols.columns
    df_cols = df_cols.apply(le.fit_transform, axis=0)
    df.loc[:,cols] = df_cols
    return df

### 7. Handling Skewness in Target Variable

In [11]:
def handle_skewness(df):
    df['Item_Outlet_Sales'] = np.sqrt(df_train['Item_Outlet_Sales'])
    return df

### 8. Handling Item visibilty

In [12]:
def handle_visibilty(df):
    avg_visibility = df.pivot_table(values='Item_Visibility',index='Item_Identifier')
    miss_bool = (df['Item_Visibility'] == 0)
    df.loc[miss_bool,'Item_Visibility'] = df.loc[miss_bool,'Item_Identifier'].apply(lambda x: avg_visibility.loc[x])
    
    return df

### 8. Scaling Features

In [13]:
def scale_data(df,scaler):
    df_cols = df.columns
    
    numeric_features = df.select_dtypes(include=['float64'])
    numeric_features = [feature for feature in numeric_features if feature != 'Item_Outlet_Sales'] 
    
    
    #column_transform = df[:,numeric_features].columns
    df.loc[:, numeric_features] = scaler.fit_transform(df.loc[:, numeric_features])
    
    return df

### 9. Removing Unique Identifiers

In [14]:
def drop_ids(df,ids):
    df = df.drop(ids,axis=1)
    return df

### 10. Removing unnecessary columns and create cleaned data files

In [15]:
def remove_nonsense(df):
    data_train = df.loc[df.Source == 'train',:]
    data_test = df.loc[df.Source == 'test',:]
    
    data_train.drop(['Source'],axis=1,inplace=True)
    data_test.drop(['Item_Outlet_Sales','Source'],axis=1,inplace=True)
    
    data_train.to_csv('data/train_modified.csv')
    data_test.to_csv('data/test_modified.csv')

In [16]:
df = prepare_item_fat_content(df)
df = fill_item_weights(df)
df = find_outlet_age(df)
df = prepare_outlet_size(df)
df = add_item_type(df)

df = encode_variables(df,['Item_Fat_Content','Item_Type','Item_Type_Derived','Outlet_Type','Outlet_Identifier'])
df = convert_ordinal_variables(df,['Outlet_Location_Type','Outlet_Size'])
df = handle_visibilty(df)

df = scale_data(df,StandardScaler())
df = drop_ids(df,['Item_Identifier'])

remove_nonsense(df)


/home/mohammed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/mohammed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
